
<a href="https://colab.research.google.com/github/sndnyang/iDDPM/blob/master/iddpm_nll_bpd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# iDDPM Evaluation NLL BPD

Negative Log Likelihood, namely Bits Per Dimension

In [ ]:
# Download the pre-trained model from openAI iDDPM
!wget https://openaipublic.blob.core.windows.net/diffusion/march-2021/cifar10_uncond_50M_500K.pt

: 

In [ ]:
!pip install tensorboardX torch_fidelity

In [ ]:

# If you are use colab, you can mount your Google Drive.
# Otherwise, you need to upload the code?
import sys

sys.path.append('/drive/MyDrive/research/iDDPM')

In [ ]:

import argparse
import numpy as np
import torch.optim
import torch.utils.data
import torch.nn as nn
import torchvision.transforms as transforms

from ExpUtils import *
from utils.dataloader import datainfo, dataload
from eval_tasks import *
from iddpm.script_util import create_model_and_diffusion, model_and_diffusion_defaults

In [ ]:

arg = argparse.Namespace()
arg.dataset = 'cifar10'
arg.data_path = './data'
arg.workers = 4
arg.eval = 'nll'
arg.resume = 'cifar10_uncond_50M_500K.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
arg.device = device

: 

In [ ]:

data_info = datainfo(logger, arg)

if arg.eval == 'buffer':
    buffer = torch.load(arg.resume, map_location=arg.device)
    eval_buffer(buffer, arg)

normalize = [transforms.Normalize(mean=data_info['stat'][0], std=data_info['stat'][1])]
augmentations = transforms.Compose([
    transforms.ToTensor(),
    *normalize
])

'''
    model
'''

model, diffusion = create_model_and_diffusion(
    **model_and_diffusion_defaults()
)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of params: {format(n_parameters, ",")}')


In [ ]:

checkpoint = torch.load(arg.resume, map_location=arg.device)

# model.load_state_dict(checkpoint['model_state_dict'])
model.load_state_dict(checkpoint)
model = model.to(arg.device)

In [ ]:

def run_bpd_on_dataset(nll_model, f, loader, arg):
    all_bpd = []
    c = 0
    start = time.time()
    for images, _ in loader:
        images = images.to(arg.device)
        c += images.shape[0]
        minibatch_metrics = nll_model.calc_bpd_loop(f, images, clip_denoised=True)
        total_bpd = minibatch_metrics["total_bpd"]
        total_bpd = total_bpd.mean()
        all_bpd.append(total_bpd.item())
        if c % 100 == 0:
            print(f'{c} bpd: {total_bpd.item()}')
    bpd = np.mean(all_bpd)
    end = time.time()
    print(f"done {c} samples: bpd={bpd}, takes {end - start}")
    return bpd



In [ ]:

# https://github.com/openai/guided-diffusion/blob/main/guided_diffusion/gaussian_diffusion.py#L709
from iddpm.gaussian_diffusion import GaussianDiffusion as NllDiffusion, LossType, ModelVarType, ModelMeanType

nll_model = NllDiffusion(
    betas=diffusion.betas,
    model_mean_type=ModelMeanType.EPSILON,
    model_var_type=ModelVarType.LEARNED,
    loss_type=LossType.MSE,
    rescale_timesteps=False,
)

train_set, val_dataset = dataload(arg, augmentations, normalize, data_info)

# make sure that batch size doesn't matter, since the model use laynorm, not batch norm. small batch size is very slow.
train_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=False, pin_memory=True, num_workers=arg.workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=False, pin_memory=True, num_workers=arg.workers)


In [ ]:

val_bpd = run_bpd_on_dataset(nll_model, model, val_loader, arg)

print(f'test bpd {val_bpd}')


In [ ]:

train_bpd = run_bpd_on_dataset(nll_model, model, train_loader, arg)
print(f'train bpd {train_bpd}, test bpd {val_bpd}')